<a href="https://colab.research.google.com/github/kkimlee/TensorFlow/blob/main/%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C%EC%9A%B0%20%EA%B0%9C%EB%85%90%20%EA%B3%B5%EB%B6%80/%EB%B3%80%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

# 변수 만들기

변수는 tf.Variable을 이용해 생성

In [4]:
my_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
my_variable = tf.Variable(my_tensor)

boo_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 6+1j])

변수는 텐서처럼 작동하며, tf.Tensor에서 지원되는 데이터 구조임. dtype과 모양을 가지며 NumPy형태로 변환 가능함

In [6]:
print("shape: ", my_variable)
print("DType: ", my_variable.dtype)
print("As Numpy: ", my_variable.numpy)

shape:  <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>
DType:  <dtype: 'float32'>
As Numpy:  <bound method BaseResourceVariable.numpy of <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>>


변수는 재수성할 수 없고, 대부분의 텐서 연산은 변수에 적용 가능함

In [7]:
print("A variable:",my_variable)
print("\nViewed as a tensor:", tf.convert_to_tensor(my_variable))
print("\nIndex of highest value:", tf.argmax(my_variable))

# This creates a new tensor; it does not reshape the variable.
print("\nCopying and reshaping: ", tf.reshape(my_variable, ([1,4])))

A variable: <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

Viewed as a tensor: tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)

Index of highest value: tf.Tensor([1 1], shape=(2,), dtype=int64)

Copying and reshaping:  tf.Tensor([[1. 2. 3. 4.]], shape=(1, 4), dtype=float32)


변수는 텐서에 의해 지원되au, tf.Varialbe.assign을 사용하여 텐서를 재할당 하는것이 가능함. assign을 호출하는 경우 새로운 텐서를 할당하는 것이 아닌, 기존 텐서의 메모리가 재사용되는 방식임

In [8]:
a = tf.Variable([2.0, 3.0])
a.assign([1, 2])
try:
  a.assign([1.0, 2.0, 3.0])
except Exception as e:
  print(f"{type(e).__name__}: {e}")

ValueError: Cannot assign to variable Variable:0 due to variable shape (2,) and value shape (3,) are incompatible


기존 변수를 이용하여 새 변수를 만들면 지원 텐서가 복제됨. 두 변수는 같은 메모리를 공유하지 않음

In [9]:
a = tf.Variable([2.0, 3.0])
b = tf.Variable(a)
a.assign([5, 6])

print(a.numpy())
print(b.numpy())

print(a.assign_add([2, 3]).numpy())
print(a.assign_sub([7, 9]).numpy())

[5. 6.]
[2. 3.]
[7. 9.]
[0. 0.]


# 수명주기, 이름 지정 및 감시

tf.Variable 인스턴스는 다른 Python 객체와 같은 수명 주기를 가짐. 변수에 대한 참조가 없으면 자동으로 할당이 해체됨.

변수를 추적하고 디버그하는 데 도움이 되는 변수의 이름을 지정하는 것이 가능함. 두 변수에 같은 이름을 지정하는 것 또한 가능함

In [11]:
a = tf.Variable(my_tensor, name="Mark")
b = tf.Variable(my_tensor + 1, name="Mark")

print(a == b)

tf.Tensor(
[[False False]
 [False False]], shape=(2, 2), dtype=bool)


모델을 저장하거나 로드할 때 변수의 이름은 유지됨. 모델의 변수는 고유한 변수 이름이 자동으로 지정됨

그래디언트가 필요하지 않은 변수의 경우, 변수 생성시 trainable 옵션을 False로 설정하여 변수의 그래디언트를 사용하지 않을 수 있음

In [12]:
step_counter = tf.Variable(1, trainable=False)

# 변수 및 텐서 배치

대부분의 변수는 GPU가 사용한 경우 GPU에 배치됨

GPU가 사용 가능한 경우에도 텐서와 변수를 CPU에 배치하는 방법이 있음. 기기 배치 로깅을 이용하면 변수가 어디에 배치되어있는지 확인하는 것 또한 가능함

In [13]:
with tf.device('CPU:0'):

  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
  c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


한 기기에서 변수 또는 텐서의 위치를 설정하고 다른 기기에서 계산을 수행하는 것이 가능함. 이 경우, 기기 간에 데이터를 복사해야 하기 때문에 지연이 발생함

In [15]:
with tf.device('CPU:0'):
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.Variable([[1.0, 2.0, 3.0]])

with tf.device('GPU:0'):
  k = a * b

print(k)

tf.Tensor(
[[ 1.  4.  9.]
 [ 4. 10. 18.]], shape=(2, 3), dtype=float32)


tf.config.set_soft_device_placement는 기본적으로 켜져 있기 때문에 GPU가 없는 기기에서 이 코드를 실행하여도 코드는 실행되며, 곱셈은 GPU가 아닌 CPU에서 작동함